In [1]:
import pandas as pd
import re
import pickle

In [2]:
with open('../data/block_shapes/manhattan_coffee_count.pk','rb') as read_file:
    block_coffee_dict_for_df = pickle.load(read_file)

df_coffee_count = pd.DataFrame.from_dict(block_coffee_dict_for_df, orient='index')

with open('../data/block_shapes/manhattan_business_count.pk','rb') as read_file:
    business_count_for_df = pickle.load(read_file)
df_business_count = pd.DataFrame.from_dict(business_count_for_df, orient='index')
df_bus_coffee = pd.merge(df_business_count, df_coffee_count, left_index=True, right_index=True)
df_bus_coffee.rename(columns = {'0_x':'business_per_sqmi', '0_y':'coffee_per_sqmi'}, inplace=True)

with open('closest_el_to_block.p', 'rb') as read_file:
    dict_closest_el = pickle.load(read_file)
    
df_closest_el = pd.DataFrame.from_dict(dict_closest_el, orient='index')
df_bus_coff_el = pd.merge(df_bus_coffee, df_closest_el, left_index=True, right_index=True)
df_bus_coff_el.rename(columns={0:'closest_el', 1:'man_dist_to_el', 2:'closest_el_daily_riders'}, inplace=True)
df_pop = pd.read_csv('../data/population_censusblock/Population_by_2010_Census_Block.csv')
df_pop_cof_bus_el = pd.merge( df_pop,df_bus_coff_el,left_on='CENSUS BLOCK',right_index=True)
df_pop_cof_bus_el.drop(columns = ['CENSUS BLOCK FULL'], inplace=True)

with open('best_coffee_block_dict.p', 'rb') as read_file:
    best_coffee_block_dict = pickle.load(read_file)

df_best_shops = pd.DataFrame.from_dict(best_coffee_block_dict, orient='index')
df = pd.merge( df_pop_cof_bus_el,df_best_shops, left_on='CENSUS BLOCK', right_index=True)

df.rename(columns={0:'best_coffee_shop'}, inplace=True)

with open('../models/dataframes/df_with_best_shops.p','wb') as write_file:
    pickle.dump(df, write_file)